In [102]:
import numpy as np
import pandas as pd
import re

In [103]:
df = pd.read_json('craigslist.json')

# remove entries with price not listed; convert price from string to int
df = df[pd.notnull(df['price'])]
df.price = df.price.str[1:].astype(int)

# remove fluff from beginning for description
df['description'] = df['description'].apply(lambda x: x[28:])

In [104]:
bed, bath, pets, laundry, parking, property_type, area, company, neighborhood, utilities, year_built = ([] for i in range(11))
for item in df.tags.values:
    room_flag, pets_flag, laundry_flag, parking_flag, property_flag, area_flag = ([True] * 6)
    pets_flag = True
    for tag in item:

        # bed, bath
        bed_bath = re.search('(\d*\.?\d)BR \/ (\d*\.?\d)Ba', tag)
        if not room_flag:
            pass
        elif bed_bath != None and room_flag:
            room_flag = False
            bed.append(bed_bath.group(1))
            bath.append(bed_bath.group(2))
        
        # pets
        pets_ = re.search('(cats)|(dogs)', tag)
        if not pets_flag:
            pass
        elif pets_ != None and pets_flag:
            pets_flag = False
            pets.append(1)
        
        # laundry
        if not laundry_flag:
            pass
        laundry_ = re.search('(?<!no\s)laundry|(w\/d)', tag)
        if laundry != None and laundry_flag:
            laundry_flag = False
            laundry.append(1)
        
        # parking 
        if not parking_flag:
            pass
        parking_ = re.search('(?<!no )parking|garage|carport', tag)
        if parking_ != None and parking_flag: 
            parking_flag = False
            parking.append(1)
        
        # area
        if not area_flag:
            pass
        area_ = re.search('(\d*)ft2', tag)
        if area_ != None and area_flag:
            area_flag = False
            area.append(int(area_.group(1)))
            
        # property_type
        if not property_flag:
            pass
        property_ = re.search('(?<! )(house|apartment|duplex|townhouse|condo)', tag)
        if property_ != None and property_flag:
            property_flag = False
            property_type.append(property_.group(1))

    if room_flag:
        bed.append(None)
        bath.append(None)
    if laundry_flag:
        laundry.append(0)
    if pets_flag:
        pets.append(0)
    if parking_flag:
        parking.append(0)
    if area_flag:
        area.append(0)
    if property_flag:
        property_type.append(None)
    company.append(None)
    neighborhood.append(None)
    year_built.append(None)
    utilities.append([])

In [105]:
df['bed'] = bed
df['bath'] = bath
df['pets'] = pets
df['laundry'] = laundry
df['parking'] = parking
df['area'] = area
df['property_type'] = property_type
df['company'] = company
df['neighborhood'] = neighborhood
df['utilities'] = utilities
df['year_built'] = year_built
df.drop('tags', axis=1)

,price,description,images,posted_datetime,address,bed,bath,pets,laundry,parking,area,property_type,company,neighborhood,utilities,year_built
0,1095,This bedroom apartments includes:\n\n→ Contemp...,[https://images.craigslist.org/00S0S_8ZkasrSZ6...,2020-01-21T10:50:37-0500,3050 Birch Hollow Dr,2,1,1,1,1,876,apartment,None,None,[],None
1,2000,"Stunning Ann Arbor home, perfect family house ...",[https://images.craigslist.org/00Y0Y_4tey2xf24...,2020-01-07T19:39:04-0500,912 Rose Ave,4,2.5,1,1,1,2000,house,None,None,[],None
2,695,"AWESOME Affordable. This stunning 3 bedroom, 1...",[],2020-01-17T14:57:45-0500,None,3,2,0,1,1,0,apartment,None,None,[],None
3,1385,Have you heard about our Stars & Stripes progr...,[https://images.craigslist.org/00p0p_1PnkFaSsW...,2020-01-08T09:49:12-0500,1505 Natalie Lane near Kipling Drive,1,1,1,1,1,800,apartment,None,None,[],None
4,1185,Have you heard about our Stars & Stripes progr...,[https://images.craigslist.org/00k0k_7n7PgfXF5...,2020-01-15T09:42:20-0500,1505 Natalie Lane near Kipling Drive,1,1,1,1,1,800,apartment,None,None,[],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1644,Orion NorthStar show contact info\n Unit D...,[https://images.craigslist.org/00Q0Q_jicpO71NG...,2019-12-23T09:49:54-0500,2820 Windwood,2,2,1,1,1,943,apartment,None,None,[],None
2996,1145,"Centrally located, spacious floorplan, huge ki...",[https://images.craigslist.org/00U0U_buQtes1a1...,2019-12-16T14:48:38-0500,553 S 4th Ave,1,1,1,1,0,0,apartment,None,None,[],None
2997,1450,Great Amenities:\n*No Smoking\n*Sparkling Swim...,[https://images.craigslist.org/00I0I_9YIVPzxaZ...,2019-12-23T09:43:36-0500,600 Hidden Valley Club Drive near State Street,2,2,1,1,0,916,apartment,None,None,[],None
2998,989,Lynden ParkeWe're located at: 2224 Golfside Dr...,[https://images.craigslist.org/00W0W_bKfxfWQrA...,2019-12-23T09:41:43-0500,None,1,1,1,1,0,658,apartment,None,None,[],None


In [106]:
col = ['address', 'price', 'bed', 'bath', 'area', 'company', 'neighborhood', 'laundry', 'pets', 'parking', 'utilities', 'property_type', 'year_built', 'description', 'images']
df = df[col]

In [107]:
df.iloc[0]

address                                       3050 Birch Hollow Dr
price                                                         1095
bed                                                              2
bath                                                             1
area                                                           876
company                                                       None
neighborhood                                                  None
laundry                                                          1
pets                                                             1
parking                                                          1
utilities                                                       []
property_type                                            apartment
year_built                                                    None
description      This bedroom apartments includes:\n\n→ Contemp...
images           [https://images.craigslist.org/00S0S_8ZkasrSZ

In [108]:
df.to_json(r'../data/craiglist_CLEANED.json')